In [2]:
import math

from itertools import product, combinations
from operator import itemgetter
from collections import defaultdict

import numpy as np
import pandas as pd
from pandas.compat import StringIO

from scipy.spatial import distance
from sklearn.metrics import mean_absolute_error

In [3]:
lo_MovieId = []
lo_movie = []

with open("data/ml-1m/movies.dat", "r", encoding='windows-1252') as infile:
    for line in infile:
        movieId, movie, _ = line.split("::")
        lo_MovieId.append(movieId)
        lo_movie.append(movie)

arr_MovieId = np.array(lo_MovieId)[:, np.newaxis]
arr_movie = np.array(lo_movie)[:, np.newaxis]
MovieId2movie = np.concatenate((arr_MovieId, arr_movie), axis=1)
        
        

In [4]:
print(MovieId2movie[:10])
len_movies = len(lo_movie)
print(len_movies)

[['1' 'Toy Story (1995)']
 ['2' 'Jumanji (1995)']
 ['3' 'Grumpier Old Men (1995)']
 ['4' 'Waiting to Exhale (1995)']
 ['5' 'Father of the Bride Part II (1995)']
 ['6' 'Heat (1995)']
 ['7' 'Sabrina (1995)']
 ['8' 'Tom and Huck (1995)']
 ['9' 'Sudden Death (1995)']
 ['10' 'GoldenEye (1995)']]
3883


In [5]:
user2movies_ratings = defaultdict(dict)
los_movieId = []

with open("data/ml-1m/ratings.dat", "r", encoding='utf-8') as infile:
    for line in infile:
        userId, movieId, rating = [int(el) for el in line.split("::")[:3]]
        user2movies_ratings[userId][movieId] = rating
        los_movieId.append(movieId)

max_movieId = max(los_movieId)
print(max_movieId)

3952


In [355]:
len_users = len(user2movies_ratings)
len_movies = max_movieId

init_matrix = np.zeros((len_users,len_movies))

for user in user2movies_ratings:
    for movie in user2movies_ratings[user]:
        review = user2movies_ratings[user][movie]
        init_matrix[user-1, movie-1] = review

all_id_means =[]
for i in range(len_users):
    all_id_means.append([i+1, init_matrix[i][init_matrix[i] != 0].mean()])
    
arridmeans = np.array(all_id_means)

matrix_and_means = np.hstack((arridmeans,init_matrix))

In [246]:
all_id_means =[]
mean = True
test = False
if test:
    for i in range(5):
        id_mean = init_matrix[i][init_matrix[i] != 0].mean()
        print(id_mean)
        all_id_means.append([i, id_mean])
        if mean:
            init_matrix[i][init_matrix[i] !=0] -= id_mean

    id_mean = np.array(all_id_means)
    print(id_mean[:5])

print(init_matrix[0][init_matrix[0] != 0])

[5. 5. 5. 4. 5. 4. 4. 4. 5. 4. 3. 3. 3. 4. 3. 4. 4. 5. 5. 5. 5. 4. 5. 3.
 4. 4. 5. 5. 4. 4. 4. 5. 4. 5. 4. 4. 5. 4. 3. 3. 5. 4. 3. 4. 4. 4. 4. 5.
 4. 5. 4. 4. 4.]


In [112]:
print(arridmeans.shape,"\n")
print(matrix_and_means[:,2:][0])

(6040, 2) 

[5. 0. 0. ... 0. 0. 0.]


In [8]:
def cosine_similarity(p,q):
    d = sum(pi * qi for pi,qi in zip(p, q))
    mag_p = math.sqrt(sum([pi**2 for pi in p]))
    mag_q = math.sqrt(sum([qi**2 for qi in q]))
    sim = d / ( mag_p * mag_q)
    return sim

In [376]:

A = np.arange(35).reshape(7,5)
indarr = np.array([0,1,2,3,4,5])
b=A!=5
bb = A[np.all(b, axis=1)]
bb[2] = np.zeros((5))
bb[4] = np.zeros(5)
bb[1][3] = 0
bb[1][2] = 0
bb[1][4] = 0

B = np.arange(10)
C = np.arange(10)[3:6]
D = np.setdiff1d(B,C)
E = np.flip(B.reshape(5,2))
sbyrow2 = np.argsort(E.T[0, :])

print(B * B)

[ 0  1  4  9 16 25 36 49 64 81]


In [ ]:
        recommendations = []
        for movie in self.targetmovies:
            weigthed_scores = []
            similarities = []
            for user, sim in self.neighbourhood:
                if movieId in self.user_ratings[user]:
                    weigthed_scores.append(sim * self.user_ratings[user][movie])
                    similarities.append(sim)
                    recommendations[movie] = sum(weigthed_scores) / sum(similarities)
        self.recommendations = recommendations

In [386]:
class KNearestNeighbour(object):
    def __init__(self, k = 50, sim_treshold = 0, measure = "cosine", show = True, mean= True):
        self.k = k
        self.sim_treshold = sim_treshold
        self.smeasure = measure
        self.fmeasure = None
        self.data = None
        self.MovieId = None
        self.targetid = None
        self.compid = None
        self.neighbourhood = None
        self.prediction = None
        self.targetmovies = None
        self.mean = mean
        self.show = show
        
        
    def init_data(self, data):
        #Take the input data matrix and add two new columns at the start, the first with the user id, the second with 
        #this user's mean. Also if we want all our data to be normalized and mean=True, we do that here as well
        new_data = data.copy()
        all_id_means =[]
        for i in range(len(new_data)):
            id_mean = new_data[i][new_data[i] != 0].mean()
            all_id_means.append([i+1, id_mean])
            if self.mean:
                new_data[i][new_data[i] !=0] -= id_mean
        
        id_mean = np.array(all_id_means)
        self.data = np.hstack((id_mean,new_data))
        
    def get_neighbourhood(self):
        #Get all the movies that your target id has rated without the movies you want to target
        targetid_movies = np.setdiff1d((np.where(self.data[self.targetid] !=0)[0] -1),self.targetmovies) + 1
        
        #Get an array of all the ratings of your target
        targetid_ratings = self.data[self.targetid][targetid_movies]
        
        #Take only the targeted movies
        only_targets = self.data.T[targetid_movies].T
        
        #Be sure to take the data from only_targets[:,2:] as everyone has their id and mean in the first two columns
        no_empty_ids = only_targets[np.any(only_targets[:,2:]!=0,axis=1)]
        not_user = no_empty_ids[no_empty_ids[:,0]!= self.targetid+1]
        
        if self.show:
            print("Shape of targetid_movies",targetid_movies.shape,"\n")
            print("Shape of targetid_ratings",targetid_ratings.shape,"\n")
            print("Shape of only_targets",only_targets.shape,"\n")
            print("Shape of no_empty_ids",no_empty_ids.shape,"\n")
            print("Shape of not_user",not_user.shape,"\n")

        #Get the Matrix of users who have at least the sim_treshold amount of ratings        
        satisfy_n = []
        for i in range(len(not_user)):
            if len(not_user[i][not_user[i] != 0]) <= self.sim_treshold:
                continue
            satisfy_n.append(not_user[i])
        final_matrix = np.array(satisfy_n)
        
        if self.show:
            print("Shape of final_matrix is:",final_matrix.shape,"\n")
        
        #Now calculate the similarity for each id that has at least sim_treshold movies in common
        sim_and_id = []
        for compid in final_matrix:
            comp_movies = np.where(compid !=0)[0]
            target_ratings = targetid_ratings[comp_movies][2:]
            comp_ratings = compid[comp_movies][2:]
            sim = self.fmeasure(target_ratings, comp_ratings)
            sim_and_id.append([compid[0], sim])

        sim_and_id = np.array(sim_and_id)
        
        if self.show:
            print("Shape of sim_and_id:",sim_and_id.shape,"\n")
            print("Example of sim_and_id is:",sim_and_id[:5],"\n")
            
        #We define our neighbourhood to be the first k highest sims, with their ID's in the first column                       
        self.neighbourhood = sim_and_id[np.flip(np.argsort(sim_and_id.T[1, :]))][:self.k]

    def get_recommendations(self):
        #Get the users who where in this neighbourhood, make the type int so we can use indexing
        top_k_users = self.neighbourhood.T[0].astype(int)
        
        #Take the movie_ratings our target had, only used if show = True
        target_movie_ratings = self.data[self.targetid][self.targetmovies+1]
        
        #Takes the matrix of neighbour_ratings, for all our k users we get their scores regarding the target_movies
        neighbour_ratings = self.data[top_k_users][:,self.targetmovies+1]

        if self.show:
            print("Targetuser has these ratings: ",target_movie_ratings,"\n")
            print("top_k_users has shape: ",top_k_users.shape,"\n")
            print("neighbour_ratings[:5] looks like: ",neighbour_ratings[:5],"\n")
        
        #Calculate the predictions for each movie
        predictions = []
        for movie in neighbour_ratings.T:
            users_rated = np.where(movie !=0)[0]
            sims_users = self.neighbourhood.T[1][users_rated]
            weighted_scores = movie[users_rated] * sims_users
            print(sum(weighted_scores),sum(sims_users))
            predict = sum(weighted_scores) / sum(sims_users)
            predictions.append(predict)
          
        predictions = np.array(predictions)  
        self.predictions = predictions
        if self.mean:
            self.predictions = predictions + self.data[self.targetid][1]
      
        if self.show:
            print("predictions looks like ",predictions)
            
    def get_ratings(self, targetid, targetmovies):
        """Combines get_neighbourhood and get_recommendations to get the final predicted score per movie"""
        #As we are mostly working with indexes subtracted the 1 from a movie ID here, makes our life easier in the future
        self.targetid = targetid - 1
        self.targetmovies = targetmovies
        
        
        self.sim_measure()
        self.get_neighbourhood()
        self.get_recommendations()
        
        #Return an array of the movies and there respected score
        return np.array([self.targetmovies,self.predictions]).T
    
          
    def euclidean_similarity(self, p, q):
        dist = math.sqrt(sum((pi-qi)**2 for pi,qi in zip(p, q)))
        sim = 1 / (1+dist)
        return sim   
        
    def manhattan_similarity(self, p, q):
        dist = sum(np.abs(pi-qi) for pi,qi in zip(p, q))
        sim = 1 / (1+dist)
        return sim    

    def cosine_similarity(self, p, q):
        d = sum(pi * qi for pi,qi in zip(p, q))
        mag_p = math.sqrt(sum([pi**2 for pi in p]))
        mag_q = math.sqrt(sum([qi**2 for qi in q]))
        sim = d / ( mag_p * mag_q)
        return sim
    
    def pearson_correlation(self, p, q):
    # this code does not scale well to large datasets. In the following, we rely on 
    # scipy.spatial.distance.correlation() to compute long vectors
        if len(self.p) > 99:
            return 1 - distance.correlation(p,q)        
        
        p_mean = sum(self.p) / len(p)
        p_deviations = [(pi-p_mean) for pi in p]
        
        q_mean = sum(self.q) / len(q)
        q_deviations = [(qi-q_mean) for qi in q]
        
        cov = sum(pde * qd for pde,qd in zip(p_deviations, q_deviations))
            
        sds_product = math.sqrt(sum((pder)**2 for pder in p_deviations) * sum((qd)**2 for qd in q_deviations))
        
        if sds_product != 0:
            r = cov / sds_product
        else:
            r = 0
        return r
    
    def jaccard_sets(self, p, q): 
        intersection_cardinality = len(set(p).intersection(set(q)))
        union_cardinality = len(set(p).union(set(q)))
        sim = intersection_cardinality / union_cardinality
        return sim
    
    def jaccard_binary(self, p, q):
        # only works for binary vectors! Binarize your vectors first
        m_11, m_01, m_10 = 0, 0, 0
        for pi, qi in zip(p, q):
    
            if pi == 1:
                if qi == 1:
                    m_11 += 1
                else:
                    m_10 += 1
                    
            elif qi == 1:
                m_01 += 1
        
        sim = m_11 / (m_10 + m_01 + m_11) 
        return sim
    
    def sim_measure(self):
        all_measures = {"euclidean" : self.euclidean_similarity,
                        "manhatten" : self.manhattan_similarity,
                        "cosine": self.cosine_similarity, 
                        "correlation": self.pearson_correlation,
                        "jaccardset" : self.jaccard_sets,
                        "jaccardbin" : self.jaccard_binary}
        if self.smeasure in all_measures.keys():
            self.fmeasure = all_measures[self.smeasure]
            

In [396]:
KNN = KNearestNeighbour(k = 50, sim_treshold = 20, measure = "cosine", show = True, mean= True)
data = KNN.init_data(init_matrix)


In [397]:
test1 = KNN.get_ratings(4447, np.array([593, 1198, 1270]))
print(test1)

Shape of targetid_movies (981,) 

Shape of targetid_ratings (981,) 

Shape of only_targets (6040, 981) 

Shape of no_empty_ids (6039, 981) 

Shape of not_user (6038, 981) 

Shape of final_matrix is: (4762, 981) 

Shape of sim_and_id: (4762, 2) 

Example of sim_and_id is: [[ 2.         -0.09280244]
 [ 3.         -0.10080074]
 [ 4.          0.13154182]
 [ 5.          0.44460952]
 [ 6.         -0.30385353]] 

Targetuser has these ratings:  [ 0.92668024 -1.07331976 -0.07331976] 

top_k_users has shape:  (50,) 

neighbour_ratings[:5] looks like:  [[ 0.          0.          1.025     ]
 [ 0.          0.          0.        ]
 [ 1.4893617   0.          0.        ]
 [ 0.71546961 -0.28453039  0.        ]
 [ 1.02745995  1.02745995  0.02745995]] 

12.387433096261582 15.784342411761571
6.468704031305176 12.317806250141079
2.2117307302096343 14.006205190639104
predictions looks like  [0.78479247 0.52515066 0.15791078]
[[ 593.            3.85811223]
 [1198.            3.59847041]
 [1270.            3

In [393]:
print(test1.T)

[[ 593.         1198.         1270.        ]
 [   3.85811223    3.59847041    3.23123053]]
